In [1]:
import numpy as np
from utils.utils import (write_json, get_data_sub_paths,
                         load_total_counts, set_system_seed)
from utils.configs import BaseConf
from utils.data_processing import sequencify
from utils.preprocessing import split_data
from torch.autograd import Variable


In [2]:
conf = BaseConf()

data_sub_paths = [i for i in get_data_sub_paths() if i.startswith('Totals')]

In [3]:
# load data
df = load_total_counts(folder_name=data_sub_paths[0])

In [4]:
df

,Total,Theft,Battery,Criminal Damage,Narcotics,Assault,Burglary,Motor Vehicle Theft,Robbery
2014-01-01 00:00:00,68,24,17,13,3,3,3,4,1
2014-01-01 01:00:00,68,22,32,5,3,5,1,0,0
2014-01-01 02:00:00,55,8,28,7,0,4,2,1,5
2014-01-01 03:00:00,30,3,21,4,0,0,0,0,2
2014-01-01 04:00:00,36,3,23,2,0,3,2,2,1
...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,26,7,2,8,2,3,0,3,1
2018-12-31 20:00:00,26,10,4,7,2,0,0,2,1
2018-12-31 21:00:00,28,4,5,12,1,0,0,3,3
2018-12-31 22:00:00,34,3,10,9,1,5,3,2,1


In [5]:
import torch
from torch import nn

In [6]:
class TimeSeriesCNN(nn.Module):
    def __init__(self):
        super(TimeSeriesCNN, self).__init__()

        self.name = "TimeSeriesCNN"
        
        
        self.layers = nn.Sequential(
            nn.Conv1d(
                in_channels=1, #: int,
                out_channels=2, #: int,
                kernel_size=5, #: Union[int, Tuple[int]],
                stride=1, #: Union[int, Tuple[int]] = 1,
                padding=0, #: Union[str, int, Tuple[int]] = 0,
                dilation=1, #: Union[int, Tuple[int]] = 1,
                groups=1, #: int = 1,
                bias=True, #: bool = True,
                padding_mode='zeros', #: str = 'zeros',
#                 device=, #=None,
#                 dtype=, #=None,
            )
        )
        
    def forward(self, x):
        y = self.layers(x)
        
        return y

In [11]:

max_seq_len = 24*7
# data = sequencify(df.Total.values,max_seq_len)
data = sequencify(np.arange(1000),max_seq_len)
train_data_fraction = 0.8


train_data, test_data = split_data(data,train_data_fraction)
train_data, val_data = split_data(train_data,train_data_fraction)
train_x, train_y = torch.tensor(train_data[:,:-1]), torch.tensor(train_data[:,-1])
test_x, test_y = torch.tensor(test_data[:,:-1]), torch.tensor(test_data[:,-1])
val_x, val_y = torch.tensor(val_data[:,:-1]), torch.tensor(val_data[:,-1])

In [12]:
model = TimeSeriesCNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.07)
criterion = nn.functional.mse_loss
    
print(model)

TimeSeriesCNN(
  (layers): Sequential(
    (0): Conv1d(1, 2, kernel_size=(5,), stride=(1,))
  )
)


In [13]:
def train(epoch):
    model.train()
    tr_loss = 0
    
    x_train, y_train = Variable(train_x), Variable(train_y)
    x_val, y_val = Variable(val_x), Variable(val_y)
    
    optimizer.zero_grad()
    
    
    output_train = model(x_train)
    output_val = model(x_val)

    
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [14]:
n_epochs = 3
train_losses = []
val_losses = []
for epoch in range(n_epochs):
    train(epoch)

RuntimeError: Expected 3-dimensional input for 3-dimensional weight [2, 1, 5], but got 2-dimensional input of size [532, 167] instead